In [1]:
import json
from math import ceil
from pathlib import Path
import csv
import json
import importlib
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import re
from lxml import etree
import nltk
from itertools import chain

OPENCORP = '/data/annot.opcorpora.xml'
RE_ENDS_WITH_PUNCT = re.compile(r".*\W$")

In [27]:
sents = list(etree.parse(OPENCORP).xpath('//source/text()'))

_monolit = []
_compound = []
for s1, s2 in zip(sents[:-1], sents[1:]):
    _monolit.append(s1.strip())
    if RE_ENDS_WITH_PUNCT.match(s1):
        _compound.append([s1.strip(), s2.strip()])
        
del sents

def check_sent_tokenizer(tokenizer):
    correct_count_mono = 0
    for m in _monolit:
        correct_count_mono += len(tokenizer(m)) == 1

    correct_count_comp = 0
    for s1, s2 in _compound:
        correct_count_comp += tokenizer(s1 + ' ' + s2) == [s1, s2]

    return correct_count_mono / len(_monolit), correct_count_comp / len(_compound)

In [3]:
%time m, c = check_sent_tokenizer(nltk.sent_tokenize)
print(f'nltk.sent_tokenizer scores: {m*100:.2f}%, {c*100:.2f}%')

CPU times: user 8.68 s, sys: 981 µs, total: 8.68 s
Wall time: 8.68 s
nltk.sent_tokenizer scores: 94.30%, 83.16%


In [4]:
%time m, c = check_sent_tokenizer(lambda s: nltk.sent_tokenize(s, 'russian'))
# donwload from https://github.com/mhq/train_punkt russian.pkl

print(f'nltk.sent_tokenizer scores: {m*100:.2f}%, {c*100:.2f}%')

CPU times: user 8.8 s, sys: 4.13 ms, total: 8.81 s
Wall time: 8.81 s
nltk.sent_tokenizer scores: 95.53%, 85.31%


In [24]:
# =================== Sentence Tokenizer ==================

def is_end(pos, par):
    return pos == len(par) - 1


def basic(position, par):
    """Заканчивается на точку, вопросительный или восклицательный знаки и пробел"""
    return re.match("^[\.\?!] $",par[position - 1 : position + 1]) is not None


def prev_capital(position, par):
    """На 2 позиции до рассматриваемой стоит заглавная буква."""
    return re.match(u"[A-ZА-Я]",par[position - 2: position - 1]) is not None


def next_capital(position, par):
    """Следующая -- заглавная (+ basic + не prev_capital)"""
    return basic(position, par) and not prev_capital(position, par) and re.match(u" [A-ZА-Я0-9]",par[position : position + 2]) is not None 


def check_shortenings(pos, par):
    """Перед точкой -- не скоращение из 1-2 букв"""
    return next_capital(pos, par) and re.match(u".* [a-zа-яA-ZА-Я]{1,2}\. $", par[pos - 5 : pos + 1]) is None


def num(pos, par):
    """После числа -- точка и пробел; кажется, это всегда признак конца предложения"""
    return re.match(u"[0-9][\.\?!]+ $",par[pos - 5 : pos + 1]) is not None


def small_point_capital(pos, par):
    """ """
    return re.match(u"[a-zа-я0-9]+[\.\?!][A-ZА-Я]$", par[pos - 3 : pos + 2]) is not None


fpatterns = [check_shortenings, is_end, num, small_point_capital]

IN = 0
OUT = 1

def analyze_paragraph(src_text):
    state = OUT
    splitting = []
    previous = 0
    text = src_text + " "
    """Перебираем все позиции в тексте"""
    for i in range(len(text)):
        if i == "\"":
            state = (state + 1) % 2
        """Если мы внутри кавычек, то не режем"""
        if state == OUT:
            for pattern in fpatterns:
                if pattern(i, text):
                    splitting += [text[previous : i + 1].rstrip(' ')]
                    previous = i + 1
                    break
    if previous == 0:
        splitting = [text]
    return splitting


    create_dataset()

In [25]:
%time m, c = check_sent_tokenizer(analyze_paragraph)
print(f'analyze_paragraph scores: {m*100:.2f}%, {c*100:.2f}%')

CPU times: user 1min 26s, sys: 3.71 ms, total: 1min 26s
Wall time: 1min 26s
analyze_paragraph scores: 97.18%, 78.93%


In [23]:
analyze_paragraph('Hi. There.')

['Hi. ', 'There. ']

In [28]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/marat/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [34]:
nltk.sent_tokenize("Предложение раз и т.д. и т. п.", 'russian')

LookupError: 
**********************************************************************
  Resource [93mpunkt[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt')
  [0m
  Searched in:
    - '/home/marat/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
    - '/home/marat/.venv_tf18/nltk_data'
    - '/home/marat/.venv_tf18/lib/nltk_data'
    - ''
**********************************************************************


In [32]:
nltk.download()

NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> в
Command 'в' unrecognized

---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> l
Packages:
  [ ] abc................. Australian Broadcasting Commission 2006
  [ ] alpino.............. Alpino Dutch Treebank
  [ ] averaged_perceptron_tagger Averaged Perceptron Tagger
  [ ] averaged_perceptron_tagger_ru Averaged Perceptron Tagger (Russian)
  [ ] basque_grammars..... Grammars for Basque
  [ ] biocreative_ppi..... BioCreAtIvE (Critical Assessment of Information
                           Extraction Syste

True

In [3]:
from pprint import pprint
from nltk.tokenize.punkt import PunktSentenceTokenizer, PunktTrainer
 
trainer = PunktTrainer()
trainer.INCLUDE_ALL_COLLOCS = True
trainer.train(' '.join(_monolit))

In [8]:
tokenizer = PunktSentenceTokenizer(trainer.get_params())
 
# Test the tokenizer on a piece of text
sentences = _compound[0][0] + ' ' + _compound[0][1]
 
print(tokenizer.tokenize(sentences))
# ['Mr. James told me Dr.', 'Brown is not available today.', 'I will try tomorrow.']
 
# View the learned abbreviations
print(tokenizer._params.abbrev_types)
# set([...])
 
# Here's how to debug every split decision
for decision in tokenizer.debug_decisions(sentences):
    pprint(decision)
    print('=' * 30)

['Сохранится ли градус дискуссии в новом сезоне?', 'Великолепная «Школа злословия» вернулась в эфир после летних каникул в новом формате.']
{'а.н', 'ю.ш', 'г.и', 'е.п', 'и.п', 'inc', 'е.в', 'h.q', 'араб', '³', 'повторов', 'бус', 'т.д', 'д.б', 'r2', 'с.и', 'пп', 'п.э', 'к.с', 'p.»', 'м.с', 'м.ю', 'сев.-зап', 'а.б', 'к.м', 'е.е', 'г.а', 'г.г', 'в.л', 'н.a', 'м.-л', '«п', 'м.з', 'и.н', '1л.д', 'л.о', 'пер', 'лат', 'm.r', 'сокр', 'рулит', 'sp', 'u.s', 'гг.р', 'т.с', 'т.а', 'е.г', '1шт', 'н.е', 'куб', 'нач', 'э.с', 'запрятаны', 'а.е', 'c»', 'накл', 'пс', 'св', 'd.n', 'fp', 'vol', 'л', 'м', 'яз', 'э', 'с.в', 'д.а', 'её»', 'изд', 'ф.м', 'познавательно', '«.eu»', 'm.b', 'j.l', 'п.е', 'п.и', 'к.,', 'в.и', 'б.в', 'ч.д', 'ул', 'p.p.p.s', '2011г', 'итал', '.bmw', '²', 'юж', 'д.в', 'б.д', 'з.д', 'б.г', 'к.н', 'и.в', 'растяжку', 'м.е', 'а.э', 'в.п', 'в.р', 'vі', 'с.х', 'ю.к', 'dr', 'а.ф', 'мн', 'т.к', 'н.м', 'дж', 'т.з', 'гл', 'н.п', 'т', '12-п', 'p.89', '3шт', 'd3', 'в.э', '«и.о.»', '–м', 'д', 'отм

IndexError: list index out of range

In [9]:
%time m, c = check_sent_tokenizer(tokenizer.tokenize)
print(f'analyze_paragraph scores: {m*100:.2f}%, {c*100:.2f}%')

CPU times: user 5.88 s, sys: 3.75 ms, total: 5.89 s
Wall time: 5.89 s
analyze_paragraph scores: 96.93%, 86.10%


In [6]:
with open('/data/russian_legal_forums.txt') as f:
    text = f.read()

In [9]:
from pprint import pprint
from nltk.tokenize.punkt import PunktSentenceTokenizer, PunktTrainer


trainer = PunktTrainer()
trainer.INCLUDE_ALL_COLLOCS = True
%time trainer.train(text[:100_000_000])

CPU times: user 1min 29s, sys: 887 ms, total: 1min 29s
Wall time: 1min 29s


In [7]:
len(text)

812952194

In [11]:
tokenizer = PunktSentenceTokenizer(trainer.get_params())
 
# Test the tokenizer on a piece of text
sentences = 'Сохранится ли градус дискуссии в новом сезоне? Великолепная «Школа злословия» вернулась в эфир после летних каникул в новом формате.'
 
print(tokenizer.tokenize(sentences))
# ['Mr. James told me Dr.', 'Brown is not available today.', 'I will try tomorrow.']
 
# View the learned abbreviations
print(tokenizer._params.abbrev_types)
# set([...])
 
# Here's how to debug every split decision
for decision in tokenizer.debug_decisions(sentences):
    pprint(decision)
    print('=' * 30)

['Сохранится ли градус дискуссии в новом сезоне?', 'Великолепная «Школа злословия» вернулась в эфир после летних каникул в новом формате.']
{'число,мес', '092,76руб', 'ньюансами', 'в.б', 'коммун', 'ворпосы', '13.03.2002г', 'к.-п', 'разжёвано', 'и.ф', 'элитная', 'б.п.в', '1.5л', 'застоем', 'м.п', 'о.н', 'в.п', 'стёкла,т.к', 'к.л', 'околесицу', '139017191000р', 'кв.n', 'д.в.и', 'разд', 'дек.2007г', '20.05.2015г', '400тыс.руб', '«ч', 'эколог', 'абракадабра', '4000т.р', 'прокатят', 'стеклить', '03.02.2011г', 'хх.хх.2013г', '02.08.2008г', 'х.з', '390тыс', 'пропиши', 'у.к', 'нач.юр.отдела', 'преступная', 'п.c', '«м', 'адв', '2.г.п', '«__»____»19__г', 'з.а.т.о', 'окружает', 'з.м', 'б.н.р', '2.пс', 'м.ш', '3750руб', 'места,т.к', 'х.й', '22.12.09г', 'дыра,людоедство', 'г.с.ю', '6.02.2004г', '72а', 'неудовл', 'ярлыком', 'ютюб', 'б.а', '23.05.2011г', 'сер', 'нез', 'зрпл', 'а.е.в', '23.10.1979г', 'мат.кап', 'серт', 'виновника-водителя', 'пы.сы', '13.10.1955г.р', '¦т', '07.06.2011г', '01.01.92г', '

IndexError: list index out of range

In [13]:
%time m, c = check_sent_tokenizer(tokenizer.tokenize)
print(f'jurist_tokenizer scores: {m*100:.2f}%, {c*100:.2f}%')

CPU times: user 5.91 s, sys: 3.99 ms, total: 5.92 s
Wall time: 5.91 s
jurist_tokenizer scores: 95.44%, 84.85%


In [6]:
# import pickle

# with open('russian_legalforums.pickle', 'wb') as f:
#     pickle.dump(tokenizer, f)

NameError: name 'tokenizer' is not defined

In [37]:
s = 'В соответствии с п. 2 ст. 36 АПК РФ "Иск к ответчикам, находящимся или проживающим на территориях разных субъектов РФ, предъявляется в арбитражный суд по месту нахождения или месту жительства одного из ответчиков".'
s = '(ст. 395 ГК РФ)'
s = '19.02.2011 в 17.00 управляя своим автомобилем ваз 2110 государственный номерной знак с****у 56rus Я следовал по ул. Уральский проспект к дому 16 по ул. Первый микрорайон, возле дома номер 5 по улице. Второй микрорайон меня начал преследовать патрульный автомобиль ГИБДД с включенным проблесковым маячком, я остановился.'

In [38]:
from nltk import sent_tokenize
print(sent_tokenize(s))

['19.02.2011 в 17.00 управляя своим автомобилем ваз 2110 государственный номерной знак с****у 56rus Я следовал по ул.', 'Уральский проспект к дому 16 по ул.', 'Первый микрорайон, возле дома номер 5 по улице.', 'Второй микрорайон меня начал преследовать патрульный автомобиль ГИБДД с включенным проблесковым маячком, я остановился.']


In [39]:
print(sent_tokenize(s, language='russian_legalforums'))

['19.02.2011 в 17.00 управляя своим автомобилем ваз 2110 государственный номерной знак с****у 56rus Я следовал по ул.', 'Уральский проспект к дому 16 по ул.', 'Первый микрорайон, возле дома номер 5 по улице.', 'Второй микрорайон меня начал преследовать патрульный автомобиль ГИБДД с включенным проблесковым маячком, я остановился.']


In [40]:
print(sent_tokenize(s, language='russian'))

['19.02.2011 в 17.00 управляя своим автомобилем ваз 2110 государственный номерной знак с****у 56rus Я следовал по ул. Уральский проспект к дому 16 по ул.', 'Первый микрорайон, возле дома номер 5 по улице.', 'Второй микрорайон меня начал преследовать патрульный автомобиль ГИБДД с включенным проблесковым маячком, я остановился.']


['19.02.2011 в 17.00 управляя своим автомобилем ваз 2110 государственный номерной знак с****у 56rus Я следовал по ул.',
 'Уральский проспект к дому 16 по ул.',
 'Первый микрорайон, возле дома номер 5 по улице.',
 'Второй микрорайон меня начал преследовать патрульный автомобиль ГИБДД с включенным проблесковым маячком, я остановился.']

In [7]:
with open('/home/marat/nltk_data/tokenizers/punkt/russian_news.pickle', 'rb') as f:
    t = pickle.load(f)

In [8]:
for a in [a for a in t._params.abbrev_types if '.' not in a]:
    print(a)

2011-14гг
1407г
91,75гр
752с
ρu
5тыс
γe
1999году
ŋò
помещ
ӡә
επ
регенсб
штутт
1348—1366
«ш
свящ
гpeйфсв
4-7-тыс
4-ступ
хлоранила
социально-эконом
15хв
19л
rejic
прмч
cв
1640г
1991—92гг
app3
ʓ
lkg
africanews
англо-сакс
ω4
/сб
bieb
„є“
удм
ф-я
ஃ
нюрнб
жди»
凌河
экспансионистов
тверск
eф
тибетск
1129-1131гг
неделикатно
abna
/н
рисунк
гс-7
170с
бентсена-мл
xв
diphone
252с
адмо
эпич
jüd
s4a
гельсингф
1921-22гг
внутриинститут
ёду
|а
/г
парчевые
阿部
янгильдин
1937г
у,
яўр
lpz
иниа
pioneer&expeditions
водн
сочин
6b—9b
75гр
южносерб
прор
к3»
$32,52/барр
જિ
agricult
йĕ
ц»
cоавт
schneid
943—945
соиск
эль-а
–s
1944-1957гг
\о
латг
10—12тыс
ꡗꡖ
басов-мл
москва-пасс
pуб
epist
263стр
санск
«мр
відс
оао»
欒
«прим
yoz
уолтером-мл
玉
„д
оксф
‎к
маносом-мл
18-00ч
14˚с
жарг
1920г
eр
н
384с
卯
sympos
«beobacht
2014-15гг
нын
'з
432с
1020—1041-гг
с1»
чжурчж
энциклопед
bifem
висбад
рейнджера-4
tzw
т
леквинадзе
рp
0,93л
陇
e/
1390г
1коринф
ehem
северо-кавказ
2014р
—д
ш2
римл
'andorra
гельмшт
м-б
ற்
19013об
ಕೌ
50долл/ба

In [ ]:
/барр /мин
\W(нет согласных)
\d+-?\w+
(окончивается на согласный, значит не аббр)
одна буква

л-з
априле-мл

In [80]:
t._params.ortho_context

defaultdict(int,
            {'катанеа': 12,
             'джузеппина': 14,
             'или': 126,
             'мария': 62,
             'иосифина': 4,
             'иисуса': 14,
             'распятого': 38,
             'г.': 126,
             'неаполь': 14,
             'италия': 46,
             'там': 126,
             'же': 126,
             'блаженная': 62,
             'римско-католической': 38,
             'церкви': 126,
             'монахиня': 110,
             'ордена': 126,
             'босых': 102,
             'кармелиток': 38,
             'родилась': 126,
             'февраля': 126,
             'года': 126,
             'в': 126,
             'неаполе': 12,
             'италии': 46,
             'семье': 126,
             'франциска': 46,
             'и': 126,
             'маркизы': 38,
             'кончетты': 4,
             'гримальди': 14,
             'кроме': 126,
             'неё': 36,
             'было': 126,
             'еще': 126,
             'д

In [61]:
%time m, c = check_sent_tokenizer(lambda s: sent_tokenize(s, language='russian_news'))
print(f'news_tokenizer scores: {m*100:.2f}%, {c*100:.2f}%')

CPU times: user 8.73 s, sys: 4.38 ms, total: 8.74 s
Wall time: 8.74 s
news_tokenizer scores: 95.18%, 84.65%


In [57]:
t._params.abbrev_types.add('ул')

In [69]:
#for tok in 'ул т.д т.п т.к'.split():
for tok in abbrs:
    if tok not in t._params.abbrev_types:
        t._params.abbrev_types.add(tok)
    else:
        print(f'{tok} is already known')

ул is already known
д is already known
г is already known
п is already known
м is already known
г is already known
к.у is already known
т.р is already known


In [63]:
'т.к' in t._params.abbrev_types

False

In [93]:
'ауоыиэяюёе'.upper()

'АУОЫИЭЯЮЁЕ'

In [38]:
# _ABRIDGED_SPLIT =      re.compile(r'[^\w](\d-?к|\d{4}г|\d*к|\d*м)\.$', re.IGNORECASE)
_POPULAR_SHORTENINGS = re.compile(r'\b(ул|тел|корп|зам|им|руб|долл|дол|коп|адм|обл|тыс|эт|сан|барр|мин|сек|русск|обр)\.$', re.IGNORECASE)
_HAS_DOT_INSIDE = re.compile(r'[\w]+\.[\w]+\.$', re.IGNORECASE)
_INITIALS = re.compile(r'\b[А-Я]{1,4}\.$')
_ONLY_CONSONANTS = re.compile(r'\b[бвгджзйклмнпрстфхцчшщ]{1,4}\.$', re.IGNORECASE)
def loose_sent_tokenize(text: str):
    sentences = []
    sents = nltk.sent_tokenize(text)
    si = 0
    processed_index = 0
    sent_start = 0
    while si < len(sents):
        s = sents[si]
        span_start = text[processed_index:].index(s) + processed_index
        span_end = span_start + len(s)
        processed_index += len(s)

        if _POPULAR_SHORTENINGS.search(s):
            pass
        elif _ONLY_CONSONANTS.search(s):
            pass
        elif _INITIALS.search(s):
            pass
        elif _HAS_DOT_INSIDE.search(s):
            pass
        else:
            if not text[sent_start: span_end].strip():
                print('empty!')
            sentences.append(text[sent_start: span_end].strip())
            sent_start = span_end
            processed_index = span_end
        si += 1
    if sent_start != len(text):
        if text[sent_start:].strip():
            sentences.append(text[sent_start:].strip())
    return sentences

print(loose_sent_tokenize('купил за 5 руб. и остался доволен.'))
print(loose_sent_tokenize('Я ему сказал и т.к. он не послушался за 500р.'))
print(loose_sent_tokenize('Ура. Ура. 500р.'))
print(loose_sent_tokenize('И.П. Павлов.'))
print(loose_sent_tokenize('И. П. Павлов.'))
print(loose_sent_tokenize('Я ему сказал: "Чтобы ничего не трогал." Но он не послушался.'))
print(loose_sent_tokenize('Нефть за $27/барр. не снится.'))


['купил за 5 руб. и остался доволен.']
['Я ему сказал и т.к. он не послушался за 500р.']
['Ура.', 'Ура.', '500р.']
['И.П. Павлов.']
['И. П. Павлов.']
['Я ему сказал: "Чтобы ничего не трогал."', 'Но он не послушался.']
['Нефть за $27/барр. не снится.']


In [33]:
%time m, c = check_sent_tokenizer(loose_sent_tokenize)
print(f'loose_sent_tokenizer scores: {m*100:.2f}%, {c*100:.2f}%')

CPU times: user 16.8 s, sys: 8.11 ms, total: 16.8 s
Wall time: 16.8 s
loose_sent_tokenizer scores: 96.33%, 84.49%


In [39]:
%time m, c = check_sent_tokenizer(loose_sent_tokenize)
print(f'loose_sent_tokenizer scores: {m*100:.2f}%, {c*100:.2f}%')

CPU times: user 17.1 s, sys: 27.9 ms, total: 17.2 s
Wall time: 17.2 s
loose_sent_tokenizer scores: 96.36%, 84.53%


In [140]:
loose_sent_tokenize('Я ему сказал: "Чтобы ничего не трогал." Но он не послушался.')

['Я ему сказал: "Чтобы ничего не трогал."', 'Но он не послушался.']

In [21]:
text = '''Чистая прибыль ООО "МЕРИДИАН" за 2015 год составила 36,00 тыс. руб. по сравнению с 14,00 тыс. руб. годом ранее.


Компания "МЕРИДИАН" по итогам за 2015 год увеличила продажи в 2,21 раза до 4,39 млн. руб. с 1,99 млн. руб. за аналогичный период прошлого года. Коммерческие расходы повысились в 2,21 раза до 4,32 млн. руб. в сравнении с 1,96 млн. руб. за аналогичный период прошлого года. Об этом говорится в отчете компании.


Компания "МЕРИДИАН" задекларировала рост прибыли от продаж за 2015 год в 2,13 раза до 64,00 тыс. руб. с 30,00 тыс. руб. годом ранее.'''

In [22]:
loose_sent_tokenize(text)

['Чистая прибыль ООО "МЕРИДИАН" за 2015 год составила 36,00 тыс. руб. по сравнению с 14,00 тыс. руб. годом ранее.',
 'Компания "МЕРИДИАН" по итогам за 2015 год увеличила продажи в 2,21 раза до 4,39 млн. руб. с 1,99 млн. руб. за аналогичный период прошлого года.',
 'Коммерческие расходы повысились в 2,21 раза до 4,32 млн. руб. в сравнении с 1,96 млн. руб. за аналогичный период прошлого года.',
 'Об этом говорится в отчете компании.',
 'Компания "МЕРИДИАН" задекларировала рост прибыли от продаж за 2015 год в 2,13 раза до 64,00 тыс. руб. с 30,00 тыс. руб. годом ранее.']

In [37]:
i = 0
for s1, s2 in _compound:
    spl = loose_sent_tokenize(s1 + ' ' + s2)
    if '...' in s1 + ' ' + s2 or '…' in s1 + ' ' + s2:
        continue
    if spl != [s1, s2]:
        print([s1, s2])
        print(spl)
        print('-'*80)
        i += 1
    if i > 1000:
        break

['Потом проект переехал с «Культуры» на НТВ.', 'Это помимо явных перемен в виде тут же появившихся рекламных блоков, отсутствовавших на «Культуре», позволило, с одной стороны, расширить круг гостей, с другой – изменить тон разговора.']
['Потом проект переехал с «Культуры» на НТВ. Это помимо явных перемен в виде тут же появившихся рекламных блоков, отсутствовавших на «Культуре», позволило, с одной стороны, расширить круг гостей, с другой – изменить тон разговора.']
--------------------------------------------------------------------------------
['Артисты, представители юмористического цеха и т.п.', 'К примеру, коллега Татьяны Толстой по шоу Первого канала «Минута славы», резидент «Комеди клаба» Гарик Мартиросян.']
['Артисты, представители юмористического цеха и т.п. К примеру, коллега Татьяны Толстой по шоу Первого канала «Минута славы», резидент «Комеди клаба» Гарик Мартиросян.']
--------------------------------------------------------------------------------
['При этом, возможно, стои

['В частности, она могла бросить в лицо своей покровительнице скомканные чулки, сопровождая это приказом:', '«Убери!']
['В частности, она могла бросить в лицо своей покровительнице скомканные чулки, сопровождая это приказом: «Убери!']
--------------------------------------------------------------------------------
['Тебе за что деньги платят?»,', 'а во время их совместного путешествия в Данию требовала отселить от неё госпожу Ратлеф, объясняя это тем, что «не привыкла спать в одной комнате с прислугой».']
['Тебе за что деньги платят?», а во время их совместного путешествия в Данию требовала отселить от неё госпожу Ратлеф, объясняя это тем, что «не привыкла спать в одной комнате с прислугой».']
--------------------------------------------------------------------------------
['Он пишет следующее:', '« Я в деталях помню, как госпожа Чайковская встретилась у меня с бывшим слугой императорского двора.']
['Он пишет следующее: « Я в деталях помню, как госпожа Чайковская встретилась у меня с б

['Теперь, сделав настоящее имя, он мог бы публиковать романы, подобные «Ангелам и демонам», каждый год, а если нанять литературных негров, то и чаще.', 'Но корень имени «Даниэль» (ивр.) — «правосудие».']
['Теперь, сделав настоящее имя, он мог бы публиковать романы, подобные «Ангелам и демонам», каждый год, а если нанять литературных негров, то и чаще.', 'Но корень имени «Даниэль» (ивр.)', '— «правосудие».']
--------------------------------------------------------------------------------
['Но корень имени «Даниэль» (ивр.) — «правосудие».', 'Между «Кодом» и «Символом» прошло шесть лет, это достойный и честный срок.']
['Но корень имени «Даниэль» (ивр.)', '— «правосудие».', 'Между «Кодом» и «Символом» прошло шесть лет, это достойный и честный срок.']
--------------------------------------------------------------------------------
['Это стоит почитать:', '«И были у него жёны: Рогнеда, которую поселил на Лыбеди, где ныне находится сельцо Предславино, от неё имел он четырёх сыновей: Изяслава,

['В 2009 году на стадионе прошла небольшая реконструкция, из-за чего его вместимость снизилась на 255 мест, до 75 957 мест..', 'По некоторым оценкам, для дальнейшего расширения стадиона — в особенности Южной трибуны, которая всё ещё состоит лишь из одного яруса — потребуются масштабные затраты в размере от 90 до 100 млн. фунтов.']
['В 2009 году на стадионе прошла небольшая реконструкция, из-за чего его вместимость снизилась на 255 мест, до 75 957 мест.. По некоторым оценкам, для дальнейшего расширения стадиона — в особенности Южной трибуны, которая всё ещё состоит лишь из одного яруса — потребуются масштабные затраты в размере от 90 до 100 млн. фунтов.']
--------------------------------------------------------------------------------
['Соперничество с «Манчестер Сити» началось раньше, ещё в 1890-е годы, и до сих пор остаётся ожесточённым и принципиальным, так как обе команды выступали в одном дивизионе на протяжении большей части своей истории.', 'Согласно данным опроса Do You Come Fro

['И нашёл видеоролик таксиста Девеша Мишры, размещённый на YouTube.', 'Самореклама на фоне Тадж-Махала в сочетании с контактным телефоном и почтой на Yahoo! собрала 32 тыс. просмотров, а Мишра стал так популярен, что его график расписан на месяц вперёд.']
['И нашёл видеоролик таксиста Девеша Мишры, размещённый на YouTube.', 'Самореклама на фоне Тадж-Махала в сочетании с контактным телефоном и почтой на Yahoo!', 'собрала 32 тыс. просмотров, а Мишра стал так популярен, что его график расписан на месяц вперёд.']
--------------------------------------------------------------------------------
['Самореклама на фоне Тадж-Махала в сочетании с контактным телефоном и почтой на Yahoo! собрала 32 тыс. просмотров, а Мишра стал так популярен, что его график расписан на месяц вперёд.', 'Кстати, воспользоваться его услугами гостям Сувира Котари не удалось — у таксиста оказался слишком плотный график.']
['Самореклама на фоне Тадж-Махала в сочетании с контактным телефоном и почтой на Yahoo!', 'собрала 

['— Я еще один помню:', '«Выходя из себя, не забудь закрыть рот».']
['— Я еще один помню: «Выходя из себя, не забудь закрыть рот».']
--------------------------------------------------------------------------------
['По информации службы « Работа.Ru » , участились случаи мошенничества с использованием персональных данных соискателей :', '« Мошенники в том числе используют имя сайта « Работа.Ru » .']
['По информации службы « Работа.Ru » , участились случаи мошенничества с использованием персональных данных соискателей : « Мошенники в том числе используют имя сайта « Работа.Ru » .']
--------------------------------------------------------------------------------
['И действительно , некоторые пользователи этого ресурса утверждают , что они получали письма следующего содержания :', '« Здравствуйте !']
['И действительно , некоторые пользователи этого ресурса утверждают , что они получали письма следующего содержания : « Здравствуйте !']
-------------------------------------------------------